<a href="https://colab.research.google.com/github/amrilio/seo/blob/main/Python_for_SEO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Google Auto Complete

In [ ]:
#!pip3 install --upgrade ecommercetools
from ecommercetools import seo
import csv
import pandas as pd
suggestions = seo.google_autocomplete("cara membuat", include_expanded=True)
print(suggestions)
df = pd.DataFrame(suggestions)
headerList = ['Term', 'Relevance']
df.to_csv('file_name.csv',header=headerList,index=False)

Sitemap Status Code Checker

In [ ]:
#!pip3 install --upgrade ecommercetools
import requests
import pandas as pd
from ecommercetools import seo
df = seo.get_sitemap('https://www.traveloka.com/sitemap/hotel_detail.xml')
urls = df['loc'].tolist()
df_output = pd.DataFrame(columns=['original_url', 'original_status', 'destination_url', 'destination_status'])
for url in urls:

    response = requests.get(url, headers={'User-Agent': 'Google Chrome'})
    row = {}

    if response.history:
        for step in response.history:
            row['original_url'] = step.url
            row['original_status'] = step.status_code
        row['destination_url'] = response.url
        row['destination_status'] = response.status_code
    else:
        row['original_url'] = response.url
        row['original_status'] = response.status_code
        row['destination_url'] = ''
        row['destination_status'] = ''

    print(row)

    df_output = df_output.append(row, ignore_index=True)
df_output.to_csv('file_name1.csv', header=True, index=False)
df_output

URL Status Code Checker in txt

In [ ]:
import requests
def get_status_code(url):
    try:
        r = requests.get(url)
        print ("Processing " + url)
        if len(r.history) > 0:
            chain = ""
            code = r.history[0].status_code
            final_url = r.url
            for resp in r.history:
                chain += resp.url + " | "
            return str(code) + '\t' + str(len(r.history)) + '\t' + chain + '\t' + final_url + '\t'
        else:
            return str(r.status_code) + '\t\t\t\t'
    except requests.ConnectionError:
        print("Error: failed to connect.")
        return '0\t\t\t\t'
input_file = 'url.txt'
output_file = 'output.txt'
with open(output_file, 'w') as o_file:
    o_file.write('URL\tStatus\tNumber of redirects\tRedirect Chain\tFinal URL\t\n')
    f = open(input_file, "r")
    lines = f.read().splitlines()
    for line in lines:
        code = get_status_code(line)
        o_file.write(line + "\t" + str(code) + "\t\n")
    f.close()

GMB collection

In [ ]:
import requests
import json
import csv
import time

# Set up the API key and search query
api_key = '' #maps api key
query = 'bengkel+di+bogor'
# Set up the URL to send a request to the Google Maps API
url = f'https://maps.googleapis.com/maps/api/place/textsearch/json?query={query}&key={api_key}&radius=1000'
results = []
next_page_token = None
# Loop through pages of results until we've reached the desired number or there are no more pages
while len(results) < 100 and (next_page_token is None or len(results) % 20 == 0):
    # If we have a next_page_token, add it to the URL
    if next_page_token is not None:
        url = f'https://maps.googleapis.com/maps/api/place/textsearch/json?pagetoken={next_page_token}&key={api_key}'
    # Send a request to the Google Maps API and retrieve the JSON response
    response = requests.get(url)
    json_response = json.loads(response.content)
    # Extract the information you need from the JSON response
    new_results = json_response['results']
    results.extend(new_results)
    # If there's a next page token, set it so we can retrieve the next page of results
    next_page_token = json_response.get('next_page_token')
    # Wait for a short time before making the next request to avoid hitting the API rate limit
    time.sleep(2)
# Remove duplicates from the results
results = list({r['place_id']: r for r in results}.values())
# Sort the results by rating and total ratings
results = sorted(results, key=lambda x: (-x['rating'], -x['user_ratings_total']))[:100]
# Set up the CSV file and write the header row
with open('bengkel_bogor.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Name', 'Address', 'Rating', 'Total Ratings', 'Phone Number', 'Website', 'Google Maps URL'])
    # Write each restaurant's information to the CSV file
    for result in results:
        name = result['name']
        address = result['formatted_address']
        rating = result.get('rating', '')
        total_ratings = result.get('user_ratings_total', '')
        place_id = result['place_id']
        google_maps_url = f'https://www.google.com/maps/place/?q=place_id:{place_id}'
        # Get the phone number and website for the place
        details_url = f'https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&key={api_key}'
        details_response = requests.get(details_url)
        details_json = json.loads(details_response.content)
        phone_number = details_json['result'].get('formatted_phone_number', '')
        website = details_json['result'].get('website', '')
        writer.writerow([name, address, rating, total_ratings, phone_number, website, google_maps_url])
    while next_page_token:
        next_url = f'https://maps.googleapis.com/maps/api/place/textsearch/json?pagetoken={next_page_token}&key={api_key}'
        next_response = requests.get(next_url)
        next_json_response = json.loads(next_response.content)
        next_results = next_json_response['results']
        next_page_token = next_json_response.get('next_page_token', None)
        for result in next_results:
            name = result['name']
            address = result['formatted_address']
            rating = result.get('rating', '')